# Exploring Uxarray with xCDAT


In [1]:
import glob
import warnings

import numpy as np
import xarray as xr
import xcdat as xc
import uxarray as ux

In [2]:
data_dir = "/p/user_pub/work/E3SM/1_0/1950-Control-21yrContHiVol-HR/0_25deg_atm_18-6km_ocean/atmos/native/model-output/mon/ens1/v1/"
data_paths = glob.glob(data_dir + "*.nc")

grid_path = "/p/user_pub/e3sm/grids_maps/grids/ne120.g"

## 1. Open file as a Uxarray Dataest object

A `xarray.Dataset-like`, multi-dimensional, in memory, array database. Inherits from `xarray.Dataset` and has its own unstructured grid-aware dataset operators and attributes through the uxgrid accessor.

Notes:

- [uxarray.Dataset source code](https://github.com/UXARRAY/uxarray/blob/main/uxarray/core/dataset.py#L22)
- [uxarray.Dataset documentation](https://uxarray.readthedocs.io/en/latest/user_api/generated/uxarray.UxDataset.html)


In [5]:
with warnings.catch_warnings():
    warnings.simplefilter(action="ignore", category=FutureWarning)
    uxds = ux.open_mfdataset(grid_path, data_paths[0:2])

In [6]:
uxds

<xarray.UxDataset> Size: 33GB
Dimensions:              (time: 2, ncol: 777602, cosp_prs: 7, nbnd: 2,
                          cosp_tau: 7, cosp_scol: 10, cosp_ht: 40, cosp_sr: 15,
                          cosp_sza: 5, cosp_htmisr: 16, cosp_tau_modis: 6,
                          lev: 72, ilev: 73)
Coordinates:
  * cosp_prs             (cosp_prs) float64 56B 900.0 740.0 620.0 ... 245.0 90.0
  * cosp_tau             (cosp_tau) float64 56B 0.15 0.8 2.45 ... 41.5 219.5
  * cosp_scol            (cosp_scol) int32 40B 1 2 3 4 5 6 7 8 9 10
  * cosp_ht              (cosp_ht) float64 320B 240.0 720.0 ... 1.896e+04
  * cosp_sr              (cosp_sr) float64 120B 0.605 2.1 ... 539.5 1.004e+03
  * cosp_sza             (cosp_sza) float64 40B 0.0 15.0 30.0 45.0 60.0
  * cosp_htmisr          (cosp_htmisr) float64 128B -99.0 0.25 ... 16.0 58.0
  * cosp_tau_modis       (cosp_tau_modis) float64 48B 0.8 2.45 ... 5.003e+04
  * lev                  (lev) float64 576B 0.1238 0.1828 0.2699 ... 993.8 998.5
  * ilev                 (ilev) float64 584B 0.1 0.1477 0.218 ... 997.0 1e+03
  * time                 (time) float64 16B 2.105e+04 2.725e+04
Dimensions without coordinates: ncol, nbnd
Data variables: (12/462)
    lat                  (time, ncol) float64 12MB dask.array<chunksize=(1, 777602), meta=np.ndarray>
    lon                  (time, ncol) float64 12MB dask.array<chunksize=(1, 777602), meta=np.ndarray>
    area                 (time, ncol) float64 12MB dask.array<chunksize=(1, 777602), meta=np.ndarray>
    cosp_prs_bnds        (time, cosp_prs, nbnd) float64 224B dask.array<chunksize=(1, 7, 2), meta=np.ndarray>
    cosp_tau_bnds        (time, cosp_tau, nbnd) float64 224B dask.array<chunksize=(1, 7, 2), meta=np.ndarray>
    cosp_ht_bnds         (time, cosp_ht, nbnd) float64 1kB dask.array<chunksize=(1, 40, 2), meta=np.ndarray>
    ...                   ...
    soa_c1DDF            (time, ncol) float32 6MB dask.array<chunksize=(1, 777602), meta=np.ndarray>
    soa_c1SFWET          (time, ncol) float32 6MB dask.array<chunksize=(1, 777602), meta=np.ndarray>
    soa_c2DDF            (time, ncol) float32 6MB dask.array<chunksize=(1, 777602), meta=np.ndarray>
    soa_c2SFWET          (time, ncol) float32 6MB dask.array<chunksize=(1, 777602), meta=np.ndarray>
    soa_c3DDF            (time, ncol) float32 6MB dask.array<chunksize=(1, 777602), meta=np.ndarray>
    soa_c3SFWET          (time, ncol) float32 6MB dask.array<chunksize=(1, 777602), meta=np.ndarray>

## Check Bounds Exist

Using xCDAT's `.bounds` accessor class


In [8]:
uxds.bounds.keys

['cosp_ht_bnds',
 'cosp_htmisr_bnds',
 'cosp_prs_bnds',
 'cosp_sr_bnds',
 'cosp_tau_bnds',
 'cosp_tau_modis_bnds',
 'time_bnds']

## Check the CF attributes

xCDAT requires CF attributes to be set on axes in order for the APIs
to map to them for operations. For example, the time axis should have `axis="T"` or
`axis="standard_name`".


In [13]:
uxds.cf

Coordinates:
             CF Axes: * Z: ['ilev', 'lev']
                      * T: ['time']
                        X, Y: n/a

      CF Coordinates: * vertical: ['ilev', 'lev']
                      * time: ['time']
                        longitude, latitude: n/a

       Cell Measures:   area, volume: n/a

      Standard Names: * atmosphere_hybrid_sigma_pressure_coordinate: ['ilev', 'lev']

              Bounds:   n/a

       Grid Mappings:   n/a

Data Variables:
       Cell Measures:   area, volume: n/a

      Standard Names:   n/a

              Bounds:   T: ['time_bnds']
                        cosp_ht: ['cosp_ht_bnds']
                        cosp_htmisr: ['cosp_htmisr_bnds']
                        cosp_prs: ['cosp_prs_bnds']
                        cosp_sr: ['cosp_sr_bnds']
                        cosp_tau: ['cosp_tau_bnds']
                        cosp_tau_modis: ['cosp_tau_modis_bnds']
                        time: ['time_bnds']

       Grid Mappings:   n/a

## Decode the time coordinates

- `ux.open_dataset()` and `ux.open_mfdataset()` set `decode_times=False`.
- An extra call to `xr.decode_cf()` is required to decode time coordinates before
  using xCDAT temporal averaging APIs.


In [11]:
uxds = xr.decode_cf(uxds, decode_times=True, use_cftime=True)

In [13]:
uxds["T"].time

<xarray.DataArray 'time' (time: 2)> Size: 16B
array([cftime.DatetimeNoLeap(58, 9, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(75, 9, 1, 0, 0, 0, 0, has_year_zero=True)],
      dtype=object)
Coordinates:
  * time     (time) object 16B 0058-09-01 00:00:00 0075-09-01 00:00:00
Attributes:
    long_name:  time
    bounds:     time_bnds

## Temporal Averaging using xCDAT


In [19]:
uxds.temporal.average("T")["T"]

<xarray.DataArray 'T' (lev: 72, ncol: 777602)> Size: 448MB
dask.array<truediv, shape=(72, 777602), dtype=float64, chunksize=(72, 777602), chunktype=numpy.ndarray>
Coordinates:
  * lev      (lev) float64 576B 0.1238 0.1828 0.2699 ... 986.2 993.8 998.5
Dimensions without coordinates: ncol
Attributes:
    mdims:         9
    units:         K
    long_name:     Temperature
    cell_methods:  time: mean
    operation:     temporal_avg
    mode:          average
    freq:          year
    weighted:      True

In [20]:
uxds.temporal.group_average("T", freq="day")["T"]

<xarray.DataArray 'T' (time: 2, lev: 72, ncol: 777602)> Size: 896MB
dask.array<truediv, shape=(2, 72, 777602), dtype=float64, chunksize=(1, 72, 777602), chunktype=numpy.ndarray>
Coordinates:
  * lev      (lev) float64 576B 0.1238 0.1828 0.2699 ... 986.2 993.8 998.5
  * time     (time) object 16B 0058-09-01 00:00:00 0075-09-01 00:00:00
Dimensions without coordinates: ncol
Attributes:
    mdims:         9
    units:         K
    long_name:     Temperature
    cell_methods:  time: mean
    operation:     temporal_avg
    mode:          group_average
    freq:          day
    weighted:      True

In [21]:
uxds.temporal.climatology("T", freq="day")["T"]

<xarray.DataArray 'T' (time: 1, lev: 72, ncol: 777602)> Size: 448MB
dask.array<truediv, shape=(1, 72, 777602), dtype=float64, chunksize=(1, 72, 777602), chunktype=numpy.ndarray>
Coordinates:
  * lev      (lev) float64 576B 0.1238 0.1828 0.2699 ... 986.2 993.8 998.5
  * time     (time) object 8B 0001-09-01 00:00:00
Dimensions without coordinates: ncol
Attributes:
    mdims:         9
    units:         K
    long_name:     Temperature
    cell_methods:  time: mean
    operation:     temporal_avg
    mode:          climatology
    freq:          day
    weighted:      True

In [24]:
uxds.temporal.departures("T", freq="day")["T"]

<xarray.DataArray 'T' (time: 2, lev: 72, ncol: 777602)> Size: 896MB
dask.array<sub, shape=(2, 72, 777602), dtype=float64, chunksize=(1, 72, 777602), chunktype=numpy.ndarray>
Coordinates:
  * lev      (lev) float64 576B 0.1238 0.1828 0.2699 ... 986.2 993.8 998.5
  * time     (time) object 16B 0058-09-01 00:00:00 0075-09-01 00:00:00
Dimensions without coordinates: ncol
Attributes:
    mdims:         9
    units:         K
    long_name:     Temperature
    cell_methods:  time: mean
    operation:     temporal_avg
    mode:          departures
    freq:          day
    weighted:      True